In [1]:
from pathlib import Path
from IPython.display import Video
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation
plt.rcParams['figure.figsize'] = [10, 4]
#%matplotlib inline

import MDAnalysis as mda
from MDAnalysis.analysis import rms, diffusionmap, align

import numpy as np
import pandas as pd
from tqdm import tqdm
from numba import njit, prange, jit
from numba_progress import ProgressBar
import cv2 as cv
from scipy.ndimage import uniform_filter1d 

In [5]:
# set verbose options
run_analysis_verb = False # run analysis or not
save_verb = True # save results images or not
plot_verb = False # display results images or not

# select run to analyze: 1, 2 or 3 (wt) 
run = 3

if run == 1:
    data_path = 'data/run_1/'
    results_path = 'results/teo_results/run_1/'
    gro = '../trajectories/8DD9_run_1.gro' # starting frame of trajectory
    xtc = '../trajectories/8DD9_run_1.xtc'
    xvg = '../trajectories/run1.xvg'
if run == 2:
    data_path = 'data/run_2/'
    results_path = 'results/teo_results/run_2/'
    gro = '../trajectories/8DD9_run_2.gro' # starting frame of trajectory
    xtc = '../trajectories/8DD9_run_2.xtc'
    xvg = '../trajectories/run2.xvg'
if run == 3:
    data_path = 'data/run_wt/'
    results_path = 'results/teo_results/run_wt/'
    gro = '../trajectories/8DD9_run_wt.gro' # starting frame of trajectory
    xtc = '../trajectories/8DD9_run_wt.xtc'
    #xvg = '../trajectories/run_wt.xvg'

In [6]:
# create universe from the corresponding run files
u = mda.Universe(gro, xtc)
nFrames = len(u.trajectory)
print(f'Number of frames: {nFrames} --> {int(nFrames/20)} ns')

# ticks for imshow plot
ticks_arr = np.linspace(0, nFrames + 1, 5, dtype = int)
ticks_arr_new = np.linspace(0, (nFrames+1)/20, 5, dtype = int)

# ticks for rmsf plot
arr = np.linspace(1, 306, 10, dtype=int)

# labels plot titles 
lab = ['1', '2', 'WT']

Number of frames: 10001 --> 500 ns


In [7]:
# selections of atoms for the variour runs
if run == 1 or run == 2:
        first_chain = u.select_atoms('name CA and (index 1:4689)') 
        second_chain = u.select_atoms('name CA and (index 4811:9499)') 
        binding_sites = ('resid 41 or resid 49 or resid 135 or resid 142 or resid 144 ' \
                'or resid 163:166 or resid 172 or resid 189 or resid 192')
        binding_site1 = f'({binding_sites}) and (index 1:4689)'
        binding_site2 = f'({binding_sites}) and (index 4811:9499)'
if run == 3:
        first_chain = u.select_atoms('name CA and (index 1:4682)') 
        second_chain = u.select_atoms('name CA and (index 4683:9364)')
        chain_resid_ticks = np.linspace(1, 306, 10, dtype=int)
        binding_sites = ('resid 41 or resid 49 or resid 135 or resid 142 or resid 144 ' \
                'or resid 163:166 or resid 172 or resid 189 or resid 192')
        binding_site1 = f'({binding_sites}) and (index 1:4682)'
        binding_site2 = f'({binding_sites}) and (index 4683:9364)'

binding_site1_CA = u.select_atoms(binding_site1 + ' and (name CA)')
binding_site2_CA = u.select_atoms(binding_site2 + ' and (name CA)')

lig1 = "resid 307"
lig2 = "resid 308"
lig1_selection = u.select_atoms(lig1)
lig2_selection = u.select_atoms(lig2)

# TEMPERATURE \& PRESSURE

In [ ]:
# extract temperature and pressure from xvg file
df = pd.read_csv(xvg, header=None, delimiter="\s+")
df.columns = ['time', 'temperature', 'pressure']
df.time = df.time/1000
mean_t, std_t = np.mean(df.temperature), np.std(df.temperature)
print(f'Temperature = {mean_t}', u"\u00B1", f'{std_t}', 'K')
mean_p, std_p = np.mean(df.pressure), np.std(df.pressure)
print(f'Pressure = {mean_p}', u"\u00B1", f'{std_p}', 'bar')

In [ ]:
# perform a moving average on the temperature and pressure
wind_size = 100 # 100/20 --> 5 ns
temperature = uniform_filter1d(df.temperature, size = wind_size, mode = 'nearest')
mean_t, std_t = np.mean(temperature), np.std(temperature)
print(f'Temperature = {mean_t}', u"\u00B1", f'{std_t}', 'K')

pressure = uniform_filter1d(df.pressure, size = wind_size, mode = 'nearest')
mean_p, std_p = np.mean(pressure), np.std(pressure)
print(f'Pressure = {mean_p}', u"\u00B1", f'{std_p}', 'bar')

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize = (12, 6))
ax.plot(df.time, temperature)
ax.set(xlabel = 'Time [ns]', ylabel= 'T [K]', title = 'Temperature evolution', ylim = (309, 311))
ax.grid()
ax1.plot(df.time, pressure)
ax1.set(xlabel = 'Time [ns]', ylabel= 'P [bar]', title = 'Pressure evolution', ylim = (-60, 60))
ax1.grid()
plt.tight_layout()
if save_verb: plt.savefig(results_path + 'temperature_pressure.pdf', bbox_inches = 'tight')
if plot_verb:
    plt.show()
else:
    plt.close()

# RADIUS OF GYRATION

In [8]:
# compute radius of gyration of the binding sites
if run_analysis_verb:
    Rgyr1 = []
    Rgyr2 = []
    for ts in tqdm(u.trajectory):
        Rgyr1.append((u.trajectory.time, binding_site1_CA.radius_of_gyration()))
        Rgyr2.append((u.trajectory.time, binding_site2_CA.radius_of_gyration()))
    Rgyr1 = np.array(Rgyr1)
    Rgyr2 = np.array(Rgyr2)
    np.savetxt(data_path + 'Rgyr1.txt', Rgyr1)
    np.savetxt(data_path + 'Rgyr2.txt', Rgyr2)
else:
    try:
        Rgyr1 = np.loadtxt(data_path + 'Rgyr1.txt')
        Rgyr2 = np.loadtxt(data_path + 'Rgyr2.txt')
    except:
        print('No Rgyr data found')

In [12]:
if run != 3: 
    fig, ax = plt.subplots(figsize = (6, 4))
else:
    fig, ax = plt.subplots(figsize = (12, 4))

ax.plot(Rgyr1[:,0]/1000, Rgyr1[:,1], label = 'Binding site 1')
ax.plot(Rgyr2[:,0]/1000, Rgyr2[:,1], label = 'Binding site 2')
ax.set(xlabel = 'Time [ns]', ylabel= 'Rgyr [$\AA$]', title = f'Radius of gyration of Active Sites - Run {lab[run-1]}')
ax.grid()
ax.legend()
if run!=3: ax.set_ylim(8, 14)
else: ax.set_ylim(8, 12)
plt.tight_layout()
if save_verb: plt.savefig(results_path + 'Rgyr.pdf', bbox_inches = 'tight')
if plot_verb:
    plt.show()
else:
    plt.close()

# RMSD ANALYSIS

In [ ]:
# compute the rmsd of various selections: CA, ligands, binding sites
if run_analysis_verb:
    R = rms.RMSD(u, u, select='name CA', groupselections=[lig1, lig2, binding_site1, binding_site2], ref_frame=0).run(verbose = True)
    rmsd_df = pd.DataFrame(R.rmsd, columns=['idx', 'time', 'CA', 'lig1', 'lig2', 'pot1', 'pot2'])
    rmsd_df['time'] = rmsd_df.time / 1000 # to get time in nanoseconds
    rmsd_df.to_csv(data_path + 'rmsd.csv', index=False, float_format='%.3f')
else:
    try: rmsd_df = pd.read_csv(data_path + 'rmsd.csv')
    except: print('Error: You have to run the analysis for the first time')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 6), sharex=True)
ax1.plot(rmsd_df.time, rmsd_df.CA)
ax1.set(ylabel = 'RMSD ($\AA$)', title = 'CA RMSD')
ax1.grid()

ax2.plot(rmsd_df.time, rmsd_df.lig1, label='lig1', linewidth=0.5)
ax2.plot(rmsd_df.time, rmsd_df.lig2, label='lig2', linewidth=0.5)
ax2.set(ylabel = 'RMSD ($\AA$)', title = 'Ligands RMSD')
ax2.legend()
ax2.grid()

ax3.plot(rmsd_df.time, rmsd_df.pot1, label='pot1', linewidth=0.5)
ax3.plot(rmsd_df.time, rmsd_df.pot2, label='pot2', linewidth=0.5)
ax3.set(ylabel = 'RMSD ($\AA$)', title = 'Binding sites RMSD')
ax3.set_xlabel('Time (ns)')
ax3.legend()
ax3.grid()
plt.tight_layout()

if save_verb: plt.savefig(results_path + "rmsd/rmsd.pdf", bbox_inches = 'tight')
if plot_verb: 
    plt.show()
else: plt.close()

# RMSD MAP & CLUSTERING 

In [ ]:
# Calculate the RMSD between the two frames
@njit(nogil=True)
def compute_rmsd(tr1, tr2):
    diff = tr1 - tr2
    rmsd = np.sqrt(np.mean(diff**2))
    return rmsd

# Compute the RMSD matrix between traj1 and traj2
@njit(nogil=True, parallel=True)
def compute_pairwise_rmsd(traj1, traj2, rmsd_map, progress_proxy):    
    for i in prange(len(traj1)):
        for j in prange(len(traj2)):
            rmsd_map[i, j] = compute_rmsd(traj1[i], traj2[j])
            progress_proxy.update(1)
    return rmsd_map

## LIGAND

In [ ]:
if run_analysis_verb:
    # take positions of only the lig 1 atoms
    selection = lig1
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting lig 1 rmsd map calculation")

    # compute the rmsd map
    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_lig1 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # take positions of only the lig 2 atoms
    selection = lig2
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting lig 2 rmsd map calculation")
    # compute the rmsd map 
    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_lig2 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # save the rmsd maps
    df = pd.DataFrame(rmsd_map_lig1).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_lig1.parquet', index=False, engine = 'pyarrow')

    df = pd.DataFrame(rmsd_map_lig2).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_lig2.parquet', index=False, engine = 'pyarrow')
else:
    try:
        # import rmsd map files if run_analysis_verb is False and the analysis has arleady been run
        rmsd_map_lig1 = pd.read_parquet(data_path + 'rmsd_map/rmsd_lig1.parquet')
        rmsd_map_lig2 = pd.read_parquet(data_path + 'rmsd_map/rmsd_lig2.parquet')
    except: print('Error: You have to run the analysis for the first time')

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
im = ax.imshow(rmsd_map_lig1, cmap='viridis', vmin=0, vmax=15)
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_title("Ligand 1")

im1 = ax1.imshow(rmsd_map_lig2, cmap='viridis', vmin=0, vmax=15)
ax1.set_xlabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title("Ligand 2")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map_lig.pdf', bbox_inches='tight')
if plot_verb:   
    plt.show()
else:
    plt.close()

## BINDING SITES

In [ ]:
if run_analysis_verb:
    # take positions of only the binding site 1 CA atoms
    selection = binding_site1
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting binding site 1 rmsd map calculation")
    # compute the rmsd map 
    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_pot1 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # take positions of only the binding site 2 CA atoms
    selection = binding_site2
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting binding site 2 rmsd map calculation")
    # compute the rmsd map 
    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_pot2 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)
    
    # save the rmsd maps
    df = pd.DataFrame(rmsd_map_pot1).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_pot1.parquet', index=False, engine = 'pyarrow')

    df = pd.DataFrame(rmsd_map_pot2).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_pot2.parquet', index=False, engine = 'pyarrow')
else:
    try:
        # import rmsd map files if run_analysis_verb is False and the analysis has arleady been run
        rmsd_map_pot1 = pd.read_parquet(data_path + 'rmsd_map/rmsd_pot1.parquet')
        rmsd_map_pot2 = pd.read_parquet(data_path + 'rmsd_map/rmsd_pot2.parquet')
    except:
        print("No rmsd data found. Please run analysis first.")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
im = ax.imshow(rmsd_map_pot1, cmap='viridis', vmin=0, vmax=7)
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_title("Binding site 1")

im1 = ax1.imshow(rmsd_map_pot2, cmap='viridis', vmin=0, vmax=7)
ax1.set_xlabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title("Binding site 2")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map_pot.pdf',  bbox_inches='tight')
if plot_verb:   
    plt.show()
else:
    plt.close()

## CHAINS RMSD MAP

In [ ]:
if run_analysis_verb:
    # take positions of only the chain 1 CA atoms
    selection = 'name CA and (index 1:4690)'
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting chain1 rmsd map calculation")
    # compute the rmsd map 
    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_chain1 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # take positions of only the chain 2 CA atoms
    selection = 'name CA and (index 4811:9500)'
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting chain2 rmsd map calculation")
    # compute the rmsd map 
    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_chain2 = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # save the rmsd maps
    df = pd.DataFrame(rmsd_map_chain1).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_chain1.parquet', index=False, engine = 'pyarrow')

    df = pd.DataFrame(rmsd_map_chain2).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_chain2.parquet', index=False, engine = 'pyarrow')
else:
    try:
        # import rmsd map files if run_analysis_verb is False and the analysis has arleady been run
        rmsd_map_chain1 = pd.read_parquet(data_path + 'rmsd_map/rmsd_chain1.parquet')
        rmsd_map_chain2 = pd.read_parquet(data_path + 'rmsd_map/rmsd_chain2.parquet')
    except:
        print("No rmsd data found. Please run analysis first.")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
im = ax.imshow(rmsd_map_chain1, cmap='viridis', vmin=0, vmax=4)
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_title("Chain 1")

im1 = ax1.imshow(rmsd_map_chain2, cmap='viridis', vmin=0, vmax=4)
ax1.set_xlabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title("Chain 2")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map_chains.pdf',  bbox_inches='tight')
if plot_verb:   
    plt.show()
else:
    plt.close()

## FULL CA RMSD MAP

In [ ]:
if run_analysis_verb:
    selection = 'name CA'
    traj = []
    for ts in u.trajectory:
        traj.append(u.select_atoms(selection).positions)
    traj = np.array(traj)

    print("Now starting rmsd map calculation")
    # compute the rmsd map 
    with ProgressBar(total=len(traj)*len(traj)) as progress:
        rmsd_map_CA = compute_pairwise_rmsd(traj, traj, np.zeros((len(traj), len(traj))), progress)

    # save the rmsd maps
    df = pd.DataFrame(rmsd_map_CA).round(3)
    df.columns = [f'{i}' for i in range(nFrames)]
    df.to_parquet(data_path + 'rmsd_map/rmsd_map_CA.parquet', index=False, engine = 'pyarrow')

else:
    try:
        # import rmsd map files if run_analysis_verb is False and the analysis has arleady been run
        rmsd_map_CA = pd.read_parquet(data_path + 'rmsd_map/rmsd_map_CA.parquet')
    except:
        print("No rmsd data found. Please run analysis first.")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
im = ax.imshow(rmsd_map_CA, cmap='viridis')
ax.set_xlabel('Time [ns]')
ax.set_ylabel('Time [ns]')
ax.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax.set_yticks(ticks_arr, ticks_arr_new.astype(str))
fig.colorbar(im, label = r'RMSD ($\AA$)')
if save_verb: plt.savefig(results_path + 'rmsd/rmsd_map.pdf',  bbox_inches='tight')
if plot_verb:
    plt.show()
else: plt.close()

# RMSF ANALYSIS

## FULL CHAINS RMSF

In [ ]:
# compute rmsf for the two chains
if run_analysis_verb:
    firstChain_rmsf = rms.RMSF(first_chain, verbose = True).run().rmsf
    secondChain_rmsf = rms.RMSF(second_chain, verbose = True).run().rmsf
    rmsf_chains = pd.DataFrame({'resid': first_chain.resids, 'chain1': firstChain_rmsf, 'chain2': secondChain_rmsf})
    rmsf_chains.to_csv(data_path+'rmsf_chains.csv', index=False, float_format='%.3f')
else:
    # import rmsf files if run_analysis_verb is False and the analysis has arleady been run
    try: rmsf_chains = pd.read_csv(data_path + 'rmsf_chains.csv')
    except: print('Error: You have to run the analysis for the first time')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
ax1.plot(first_chain.resids.astype(str), rmsf_chains.chain1)
ax1.set(ylabel = 'RMSF ($\AA$)', title = 'First Chain RMSF') 
ax1.set_xticks(arr, arr.astype(str))
ax1.grid()

for i in binding_site1_CA.resids: # red vertical lines for the binding site 
    if i == 144:
        ax1.vlines(i, min(rmsf_chains.chain1), max(rmsf_chains.chain1), color='red', linestyle = 'solid', alpha=1)
    else:
        ax1.vlines(i, min(rmsf_chains.chain1), max(rmsf_chains.chain1), color='black', linestyle = 'dotted', alpha=0.8)
if run==2: ax1.set_ylim(1, 6)

ax2.plot(second_chain.resids.astype(str), rmsf_chains.chain2)
ax2.set(xlabel = 'Residue', ylabel = 'RMSF ($\AA$)', title = 'Second Chain RMSF')
for i in binding_site2_CA.resids: # red vertical lines for the binding site 
    if i == 144:
        ax2.vlines(i, min(rmsf_chains.chain2), max(rmsf_chains.chain2), color='red', linestyle = 'solid', alpha=1)
    else:
        ax2.vlines(i, min(rmsf_chains.chain2), max(rmsf_chains.chain2), color='black', linestyle = 'dotted', alpha=0.8)
ax2.grid()

if run ==2: ax2.set_ylim(1, 6)

plt.tight_layout()
if save_verb: plt.savefig(results_path + "rmsf/rmsf.pdf", dpi = 500, bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

In [ ]:
# version 2
arr = np.linspace(1, 306, 10, dtype=int)

fig, ax = plt.subplots(1, 1, figsize=(12, 3), sharex=True)
ax.plot(first_chain.resids.astype(str), rmsf_chains.chain1, label = 'First Chain')
ax.plot(second_chain.resids.astype(str), rmsf_chains.chain2, label = 'Second Chain')
ax.set(xlabel = 'Risdue', ylabel = 'RMSF ($\AA$)', title = f'Chains RMSF - Run {lab[run-1]}') 
ax.set_xticks(arr, arr.astype(str))
ax.grid()

for i in binding_site1_CA.resids: # red vertical lines for the binding site 
    if i == 144:
        ax.vlines(i, -10, 10, color='red', linestyle = 'solid', alpha=1)
    else:
        ax.vlines(i, -10, 10, color='black', linestyle = 'dotted', alpha=0.8)    
ax.set_ylim(1, 10)
ax.legend(loc = 'upper left', fontsize = 8)
if save_verb: plt.savefig(results_path + "rmsf/rmsf_v2.pdf", bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

## RMSF WINDOWED IN TIME

1 frame = 50 ps $\longrightarrow$ 1 ns = 20 frames


In [ ]:
# nFrames/20 since we save coordinates every 50 ps
print(f"number of frames: {nFrames}, corresponding to {nFrames/20} ns" )
window = 500 # 25 ns, 25*20 = 500 frames
stride = 20 # 1 ns, 5*20 = 100 frames
print(f"window of {window/20} ns, stride of {stride/20} ns")

startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window

In [ ]:
if run_analysis_verb:
    rmsf_list = []
    for i in tqdm(range(len(startFrames))):
        startFrame = startFrames[i]
        endFrame = endFrames[i]
        firstChain_rmsf = rms.RMSF(first_chain).run(startFrame, endFrame).rmsf
        secondChain_rmsf = rms.RMSF(second_chain).run(startFrame, endFrame).rmsf
        rmsf_list.append(pd.DataFrame({'resid': first_chain.resids, 'chain1': firstChain_rmsf, 'chain2': secondChain_rmsf}))

    rmsf_windowed = pd.concat(rmsf_list, axis=0, ignore_index=True)
    rmsf_windowed.to_csv(data_path + 'rmsf_windowed.csv', index=False, float_format='%.3f')
else:
    try: rmsf_windowed = pd.read_csv(data_path + 'rmsf_windowed.csv')
    except: print("Error: You need to run the analysis first")

# generate video with other environment running video.py, here only show
#Video(results_path+"rmsf/windowed_rmsf.mp4")

In [ ]:
temp = np.empty((306, len(startFrames)))
for i in range(1, 307):
    temp[i-1] = np.array(rmsf_windowed.loc[rmsf_windowed.resid == i, "chain1"])

chain1_rmsf_heatmap = np.empty((306*2, len(startFrames)))
for i in range(0, 2*306, 2):
    chain1_rmsf_heatmap[i] = temp[int(i/2)]
    chain1_rmsf_heatmap[i+1] = temp[int(i/2)]

temp = np.empty((306, len(startFrames)))
for i in range(1, 307):
    temp[i-1] = np.array(rmsf_windowed.loc[rmsf_windowed.resid == i, "chain2"])

chain2_rmsf_heatmap = np.empty((306*2, len(startFrames)))
for i in range(0, 2*306, 2):
    chain2_rmsf_heatmap[i] = temp[int(i/2)]
    chain2_rmsf_heatmap[i+1] = temp[int(i/2)]

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize = (12, 5), sharex=True, sharey=True)
im = ax.imshow(chain1_rmsf_heatmap.T, cmap='hot', interpolation='nearest', vmin = 1.5, vmax = 6)
ax.set(xlabel = 'Residues', ylabel = 'Window Time [ns]', title = 'First Chain RMSF')
ax.set_xticks(2*binding_site1_CA.resids, binding_site1_CA.resids.astype(str))
plt.setp(ax.get_xticklabels(), visible=False)

im2 = ax1.imshow(chain2_rmsf_heatmap.T, cmap='hot', interpolation='nearest', vmin = 1.5, vmax = 6)

ax1.set(xlabel = 'Residues', title = 'Second Chain RMSF')
ax1.set_xticks(2*binding_site1_CA.resids, binding_site1_CA.resids.astype(str))
plt.setp(ax1.get_xticklabels(), visible=False)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD [$\AA$]')
if save_verb: plt.savefig(results_path + "rmsf/rmsf_heatmap.pdf", bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

# LIGAND DISTANCE TO BINDING SITE

In [ ]:
if run_analysis_verb:
    d_lig1 = np.zeros(nFrames)
    d_lig2 = np.zeros(nFrames)
    for i, ts in tqdm(enumerate(u.trajectory)):
        d_lig1[i] = np.linalg.norm(binding_site1_CA.center_of_mass() - lig1_selection.center_of_mass())
        d_lig2[i] = np.linalg.norm(binding_site2_CA.center_of_mass() - lig2_selection.center_of_mass())
    d_ligands = pd.DataFrame({'lig1': d_lig1, 'lig2': d_lig2})
    d_ligands.to_csv(data_path+'ligands_distance.csv', index=False, float_format='%.3f')
else:
    try: d_ligands = pd.read_csv(data_path+'ligands_distance.csv')
    except: print("Error: You need to run the analysis first")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4), sharex=True)
ax.plot(d_ligands.index/20, d_ligands.lig1, '-b')
ax.plot(d_ligands.index/20, d_ligands.lig2, '-r')
ax.axhline(y=13, color='k', linestyle='--')
ax.set(ylabel = 'Distance ($\AA$)', xlabel = 'Time [ns]', title = f'Ligand distance from corresponding binding site - Run {lab[run-1]}')
ax.grid()
plt.tight_layout()
ax.legend(['Ligand 1', 'Ligand 2', 'Threshold - 13 $\AA$ '], loc='upper left', fontsize=8)
if save_verb: plt.savefig(results_path + "ligand_distance.pdf", bbox_inches = 'tight')
if plot_verb: 
    plt.show()
else: plt.close()

# LIGAND DISTANCE FROM CA

In [ ]:
if run_analysis_verb:
    d_lig1 = np.zeros((nFrames, u.select_atoms('name CA').n_residues))
    for i, ts in tqdm(enumerate(u.trajectory)):
        for j, at in enumerate(u.select_atoms('name CA').positions):
            d_lig1[i, j] = np.linalg.norm(at - lig1_selection.center_of_mass())
    d_lig1 = pd.DataFrame(d_lig1)
    d_lig1.columns = np.arange(0, 612, 1).astype(str)
    d_lig1["Time"] = d_lig1.index/20
    d_lig1.set_index("Time", inplace=True)
    d_lig1.round(3)
    d_lig1.to_parquet(data_path + 'ligand1_distance.parquet', index=False, engine = 'pyarrow')
    
    d_lig2 = np.zeros((nFrames, u.select_atoms('name CA').n_residues))
    for i, ts in tqdm(enumerate(u.trajectory)):
        for j, at in enumerate(u.select_atoms('name CA').positions):
            d_lig2[i, j] = np.linalg.norm(at - lig2_selection.center_of_mass())
    d_lig2 = pd.DataFrame(d_lig2)
    d_lig2.columns = np.arange(0, 612, 1).astype(str)
    d_lig2["Time"] = d_lig2.index/20
    d_lig2.set_index("Time", inplace=True)
    d_lig2.round(3)
    d_lig2.to_parquet(data_path + 'ligand2_distance.parquet', index=False, engine = 'pyarrow')
    
else:
    try: 
        d_lig1 = pd.read_parquet(data_path + 'ligand1_distance.parquet')
        d_lig2 = pd.read_parquet(data_path + 'ligand2_distance.parquet')
    except: print("Error: You need to run the analysis first")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(16, 6))
im = ax.imshow(d_lig1.T, cmap = 'inferno_r', vmin = 0, vmax = 20)
ax.axhline(y=306, color='r', linestyle='--')
ax.set(xlabel = 'Time [ns]', ylabel = 'Residues', title = 'Ligand 1')
ax.set_yticks(np.append(binding_site1_CA.resids, 306+binding_site1_CA.resids ))
ax.set_xticks(np.arange(0, nFrames, 2000), np.arange(0, nFrames/20, 100, dtype=int))
ax.set_aspect(12)
plt.setp(ax.get_yticklabels(), visible=False)

im = ax1.imshow(d_lig2.T, cmap = 'inferno_r', vmin = 0, vmax = 20)
ax1.axhline(y=306, color='r', linestyle='--')
ax1.set(xlabel = 'Time [ns]', title = 'Ligand 2')
ax1.set_yticks(np.append(binding_site1_CA.resids, 306+binding_site1_CA.resids ))
ax1.set_xticks(np.arange(0, nFrames, 2000), np.arange(0, nFrames/20, 100, dtype=int))
ax1.set_aspect(12)
plt.setp(ax1.get_yticklabels(), visible=False)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.14, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'Distance ($\AA$)')
if save_verb: plt.savefig(results_path + "ligand_distance_heatmap.pdf", bbox_inches = 'tight')
plt.close()

# PLOT

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(12, 4), gridspec_kw={'width_ratios': [1.5, 1]})
ax.plot(rmsd_df.time, rmsd_df.CA)
ax.set(ylabel = 'RMSD [$\AA$]', xlabel = 'Time [ns]' , ylim = (1, 4), title = r'RMSD ($C_\alpha$)')
ax.grid()

im = ax1.imshow(rmsd_map_CA, cmap='viridis')
ax1.set_xlabel('Time [ns]')
ax1.set_ylabel('Time [ns]')
ax1.set_xticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_yticks(ticks_arr, ticks_arr_new.astype(str))
ax1.set_title(r"RMSD - Map ($C_\alpha$)")
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label = r'RMSD ($\AA$)')

if save_verb: plt.savefig(results_path + "rmsd/rmsd_CA.pdf", bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 2), sharex=True)
ax.plot(rmsd_df.time, rmsd_df.lig1, label = 'Ligand 1', linewidth=0.8)
ax.plot(rmsd_df.time, rmsd_df.lig2, label = 'Ligand 2', linewidth=0.8)
ax.set(ylabel = 'RMSD [$\AA$]', xlabel = 'Time [ns]', title = f'Ligands RMSD - Run {lab[run-1]}')
ax.legend()
ax.grid()

if save_verb: plt.savefig(results_path + "rmsd/rmsd_ligands.pdf", bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 2), sharex=True)
ax.plot(rmsd_df.time, rmsd_df.pot1, label = 'Binding Site 1', linewidth=0.8)
ax.plot(rmsd_df.time, rmsd_df.pot2, label = 'Binding Site 2', linewidth=0.8)
ax.set(ylabel = 'RMSD [$\AA$]', xlabel = 'Time [ns]', title = f'Binding Sites RMSD - Run {lab[run-1]}')
ax.legend()
ax.grid()

if save_verb: plt.savefig(results_path + "rmsd/rmsd_binding_sites.pdf", bbox_inches = 'tight')
if plot_verb:
    plt.show()
else: plt.close()